In [ ]:
import numpy as np
import random as rnd
from math import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook, tqdm
import plotly.express as px

sns.set_theme(style='white')

In [ ]:
def rnd_sign():
    return 1 if rnd.random() < 0.5 else -1

In [ ]:
class Indv:         # Definicion de un individuo
    
    def __init__(self,step_size, r_infec, p_infec, p_market, box, tmax_infec,tmax_market):
        self.box = box                     # Tamapo de la region box x box
        
        self.x = self.box * (2 * rnd.random() - 1)          # Coordenadas iniciales
        self.y = self.box * (2 * rnd.random() - 1)
        
        self.step_size = step_size           #Tamaño del paso
        self.r_infec = r_infec             #Radio de infeccion
        self.p_infec = p_infec             #Probabilidad de infeccion
        self.p_market = p_market           #Probabilidad de ir al mercado
                           
        self.tmax_infec = tmax_infec       #Tiempo de duracion de la infeccion
        self.tmax_market = tmax_market     #Tiempo en el mercado(?)
        
        self.est = 'S'            # Estado S-> suceptible, I-> infectado, R-> removido
        self.xcp, ycp = 0, 0      # Coordenadas auxiliares 
        self.t = 0  
        self.t_market = 0       #Tiempo en el mercado

    def rand_walk(self):       #Caminata aleatoria
        if self.est == 'I':    
            self.t += 1        #Contador de tiempo infectado
        if self.t == self.tmax_infec:
            self.est = 'R'     #El individuo se cura
        random_sign_x = rnd_sign()
        random_sign_y = rnd_sign()
        self.x += self.step_size * random_sign_x
        self.y += self.step_size * random_sign_y
        if abs(self.x) >= self.box:
            self.x += 2 * self.step_size * (-1*random_sign_x)
        if abs(self.y) >= self.box:
            self.y += 2 * self.step_size * (-1*random_sign_y)


    def walk_market(self):        #Caminata aleatoria pero existe la probabilidad de que 
                                  #el individuo visite un "mercado", que es un foco de infeccion
        if rnd.random() <= self.p_market and self.t_market == 0:
            self.xcp, self.ycp = self.x, self.y
            self.x, self.y = 0, 0
            self.t_market += 1
        elif self.t_market < self.tmax_market and self.t_market != 0:
            self.t_market += 1
        elif self.t_market == self.tmax_market:
            self.x, self.y = self.xcp, self.ycp
            self.t_market = 0
        else:
            if self.est == 'I':
                self.t += 1
            if self.t == self.tmax_infec:
                self.est = 'R'
            random_sign_x = rnd_sign()
            random_sign_y = rnd_sign()
            self.x += self.step_size * random_sign_x
            self.y += self.step_size * random_sign_y
            if abs(self.x) >= self.box:
                self.x += 2 * self.step_size * (-1*random_sign_x)
            if abs(self.y) >= self.box:
                self.y += 2 * self.step_size * (-1*random_sign_y)

In [ ]:
class Population:
    
    def __init__(self, N, I0, box, step_size = 0.05, r_infec = 0.05, p_infec = 0.1, p_market = 0.002, tmax_infec = 100,tmax_market = 4, rnd_starting_infection_time = False, market = False):
        
        # Simulation parameters
        
        self.N = N           # Tamaño de poblacion
        self.I0 = I0            # Infectados iniciales   
        
        # Variables for Indv class 
        self.box = box 
        self.step_size = step_size           #Tamaño del paso
        self.r_infec = r_infec             #Radio de infeccion
        self.p_infec = p_infec             #Probabilidad de infeccion
        self.p_market = p_market           #Probabilidad de ir al mercado
                           
        self.tmax_infec = tmax_infec       #Tiempo de duracion de la infeccion
        self.tmax_market = tmax_market 
        
        # Data record
        self.Su = np.zeros(0, dtype=int)  # Desarrollo de poblacion suceptible a lo largo del tiempo
        self.In = np.zeros(0, dtype=int)  # Desarrollo de poblacion infectada a lo largo del tiempo
        self.Re = np.zeros(0, dtype=int)
        
        # Special features
        self.rnd_starting_infection_time = rnd_starting_infection_time
        self.market = market
        
        #setup
        self.pop = self.populate()
        
    def populate(self):
            
        pop = [Indv(step_size = self.step_size,
                    r_infec = self.r_infec, 
                    p_infec = self.r_infec, 
                    p_market = self.p_market, 
                    box = self.box, 
                    tmax_infec = self.tmax_infec,
                    tmax_market = self.tmax_market) for i in range(self.N)] #Creacion de la poblacion
            
        #Asignacion de estatus I a I_O personas
        count = 0
        for person in pop:   
            if self.rnd_starting_infection_time:
                if count < self.I0:                  
                    person.t = rnd.randint(0, self.tmax_infec)
                    person.est = 'I'
                    count +=1
            else: 
                if count < self.I0:                
                    person.est = 'I'
                    count +=1
        
        return pop
    
    #one instance of time
    
    def be(self):
        self.Su = np.append(self.Su, 0)
        self.In = np.append(self.In, 0)
        
        for person in self.pop:
            if self.market:
                person.walk_market()
            else:
                person.rand_walk()
            if person.est == 'S':
                for other_person in self.pop:
                    if person.est != 'S':
                        break
                    elif other_person.est == 'I' and rnd.random() <= self.p_infec and sqrt(
                            (person.x - other_person.x) ** 2 + (person.y - other_person.y) ** 2) <= self.r_infec:
                        person.est = 'I'
            if person.est == 'S':
                self.Su[-1] += 1
            elif person.est == 'I':
                self.In[-1] += 1
        self.Re = np.append(self.Re, self.N - self.Su[-1] - self.In[-1])

In [ ]:
model = Population(N=3000, 
                   I0=10, 
                   box=1, 
                   tmax_infec=10)

In [ ]:
tmax = 200
for step in tqdm(range(tmax)):
    model.be()
    if model.In[-1] == 0:
        print('Early stopping. The number of infected dropped to 0.')
        break

In [ ]:
P = pd.DataFrame()
P['X'] = [person.x for person in model.pop]
P['Y'] = [person.y for person in model.pop]
P['State'] = [person.est for person in model.pop]

In [ ]:
plt.figure(figsize=(12,8))
sns.scatterplot (data=P, x='X', y='Y', hue='State', palette='magma')
plt.legend(bbox_to_anchor=(1.05, 0.5))
plt.xlim([-model.box, model.box])
plt.ylim([-model.box, model.box])

In [ ]:
lines = pd.DataFrame()
lines['Suceptible'] = model.Su
lines['Infected'] = model.In
lines['Removed'] = model.Re

In [ ]:
fig = px.line(lines)
fig.show()

In [ ]:
print('Porcentaje de infeccion: ' + str((P.groupby('State').count()['X']['R']/model.N)*100))